In [1]:
!python -m pip install cn_clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.5/306.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00


In [2]:
import os
os.chdir("/content/drive/MyDrive/workspace")

In [3]:
import json
import torch
from PIL import Image

import cn_clip.clip as clip
from cn_clip.clip import tokenize, load_from_name

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, processer = load_from_name("ViT-H-14", device=device)

100%|█████████████████████████████████████| 3.57G/3.57G [01:47<00:00, 35.5MiB/s]


Loading vision model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/ViT-H-14.json
Loading text model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/RoBERTa-wwm-ext-large-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': 32, 'vision_width': 1280, 'vision_head_width': 80, 'vision_patch_size': 14, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 1024, 'text_initializer_range': 0.02, 'text_intermediate_size': 4096, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 16, 'text_num_hidden_layers': 24, 'text_type_vocab_size': 2}


In [ ]:
model.eval()

base_dir = "haruhi_data/images_28"

info_dir = "haruhi_data/info_28.json"

data = []

with open(info_dir, 'r', encoding='utf-8') as f:
    info = json.load(f)

for item in info:
    image = processer(Image.open(f"{base_dir}/{item['path']}")).unsqueeze(0).to(device)
    text  = tokenize([item['text']]).to(device)

    with torch.no_grad():
        image_embedding = model.encode_image(image)
        text_embedding  = model.encode_text(text)

        image_embedding /= image_embedding.norm(dim=-1, keepdim=True)
        text_embedding  /= text_embedding.norm(dim=-1, keepdim=True)

    item['text_embedding'] = text_embedding
    item['image_embedding'] = image_embedding

    data.append(item)

with open("haruhi_data/info_28_embedding.json") as f:
    json.dump(data,f,ensure_ascii=False,indent=4)